In [1]:
import os
import random
import numpy as np
from PIL import Image
import shutil

In [2]:
# traing_data_dir
data_dir = 'crops_data'

In [3]:
# original_data_dir
dir = 'crops_image'

# Spiltting the images into training and testing dataset

In [4]:
# classes = {}

# for folder in os.listdir(dir):
#     image_path = os.path.join(dir, folder) 
#     images = []
#     for image in os.listdir(image_path):
#         if image.endswith('.PNG') or image.endswith('.png'):
#             images.append(image)
        
#     classes[folder] = images

In [5]:
# # 將原始資料複製到新的資料夾，並分成train,test 兩個資料夾

# print(os.getcwd())
# for key in classes.keys():
#     random.shuffle(classes[key])

#     # train : test = 8 : 2 
#     point1 = int(len(classes[key])*0.8)
    
#     for name in ['train', 'test']:
#         save_path = os.path.join(data_dir, name, key)
#         os.makedirs(save_path, exist_ok=True)
        
#         if name == 'train':
#             for image in classes[key][:point1]:
#                 src = os.path.join(dir, key, image)
#                 dst = os.path.join(save_path, image)
#                 shutil.copy(src, dst)

#         if name == 'test':
#             for image in classes[key][point1:]:
#                 src = os.path.join(dir, key, image)
#                 dst = os.path.join(save_path, image)
#                 shutil.copy(src, dst)

# Training Model

In [6]:
import torch
from torchvision.models import resnet18,resnet34,resnet50,resnet101,resnet152
import torchvision
print(torch.cuda.is_available())
from torchvision import transforms
from torchvision.datasets import ImageFolder

True


In [7]:
model = resnet50(pretrained=True)
# model = resnet18(pretrained=True)
# model = resnet34(pretrained=True)
# model = resnet101(pretrained=True)
# model = resnet152(pretrained=True)

### Data Augumentation


In [8]:
# Set input image size
input_size = (224, 224)
 
# Set the data directory


# Define data transformation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.5, contrast=0, saturation=0, hue=0),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
    ]),
}

## Data loader

In [9]:
# prevent error
def custom_loader(path):
    try:
        return Image.open(path).convert('RGB')
    except Exception as e:
        print(f"Error loading image {path}: {e}")
        # Return a blank image or handle it in a way that suits your needs
        # we turn the wrong picture into a completely white image 
        return Image.new('RGB', (224, 224), (255, 255, 255))

In [10]:
def load_data(data_dir):
    image_datasets = {x: ImageFolder(os.path.join(data_dir, x),
                        transform=data_transforms[x],loader=custom_loader) for x in ['train', 'test']}

    return image_datasets

In [11]:
print(load_data(data_dir))

{'train': Dataset ImageFolder
    Number of datapoints: 2700
    Root location: crops_data/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               ColorJitter(brightness=[0.5, 1.5], contrast=None, saturation=None, hue=None)
               ToTensor()
           ), 'test': Dataset ImageFolder
    Number of datapoints: 660
    Root location: crops_data/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )}


In [12]:
import torch.nn as nn
import torch.optim as optim

In [13]:
print(torchvision.__version__)

0.5.0


In [14]:
!nvidia-smi

Tue Dec 19 23:23:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   41C    P8    17W / 280W |     10MiB / 11177MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   35C    P8     9W / 280W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

## Train function

In [15]:
save_path = '/home/nas/Research_Group/Personal/Hamson/DS_code'

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

def train(config, model):
    
    p = os.path.join(save_path, 'checkpoint_epoch100')
    os.makedirs(p, exist_ok=True)
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
    model.to(device)

    # set the criterion, using the cross-entropy as the loss function
    criterion = nn.CrossEntropyLoss()
    # Choose a optimizer you want (e.g. Adam...)
    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9) 

    # load data
    datasets = load_data(config["data_dir"])
    dataloaders = {'train': torch.utils.data.DataLoader(datasets['train'], batch_size=int(16),
                    shuffle=True, drop_last=True, num_workers=8),
                   'test': torch.utils.data.DataLoader(datasets['test'], batch_size=int(16),
                    shuffle=True, drop_last=True, num_workers=8),}
    
    # store the accuracy
    best_acc = 0.0
    epoch = 0
    
    val_loss_history = []
    train_loss_history = []
    val_acc_history = []
    train_acc_history = []
    
    while True:
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            total = 0
            running_loss, running_corrects = 0.0, 0.0
            all_labels = []
            all_preds = []

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                total += labels.size(0)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data).item()
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

            if phase == 'train':
                train_loss = running_loss / total
                train_acc = running_corrects / total
                precision = precision_score(all_labels, all_preds, average='macro')
                recall = recall_score(all_labels, all_preds, average='macro')
                f1 = f1_score(all_labels, all_preds, average='macro')
                
            elif phase == 'test':
                val_loss = running_loss / total
                val_acc = running_corrects / total
                precision = precision_score(all_labels, all_preds, average='macro')
                recall = recall_score(all_labels, all_preds, average='macro')
                f1 = f1_score(all_labels, all_preds, average='macro')
       
        print(f'Epoch:{epoch}')
        print(f'Training accuracy: {train_acc:.4f}, training loss: {train_loss:.4f}')
        print(f'Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}')
        print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
        
        val_loss_history.append(val_loss)
        train_loss_history.append(train_loss)
        val_acc_history.append(val_acc)
        train_acc_history.append(train_acc)

        # save the best model
        if epoch == 0 or val_acc >= best_acc:
            best_acc = train_acc
            path = os.path.join(p, f'checkpoint_{epoch}.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, path)            
        
        if epoch == config["epoch"]:
            break
        else:
            epoch = epoch + 1

    print(f'Finish training! Best accuracy: {best_acc}') 
    return val_loss_history, train_loss_history, val_acc_history, train_acc_history


In [17]:
# set the training detail

training_config = {
    "lr": 0.0001,
    "epoch": 40,
    "data_dir": data_dir
}

In [18]:
# train the model
val_loss, train_loss, val_acc, train_acc = train(training_config, model)

True
Error loading image crops_data/train/atemoya/atemoya.38.png: image file is truncated
Epoch:0
Training accuracy: 0.3363, training loss: 0.2550
Validation accuracy: 0.7348, validation loss: 0.0744
Error loading image crops_data/train/atemoya/atemoya.38.png: image file is truncated
Epoch:1
Training accuracy: 0.7061, training loss: 0.0679
Validation accuracy: 0.8354, validation loss: 0.0364
Error loading image crops_data/train/atemoya/atemoya.38.png: image file is truncated
Epoch:2
Training accuracy: 0.7898, training loss: 0.0440
Validation accuracy: 0.8811, validation loss: 0.0253
Error loading image crops_data/train/atemoya/atemoya.38.png: image file is truncated
Epoch:3
Training accuracy: 0.8166, training loss: 0.0376
Validation accuracy: 0.8918, validation loss: 0.0204
Error loading image crops_data/train/atemoya/atemoya.38.png: image file is truncated
Epoch:4
Training accuracy: 0.8486, training loss: 0.0305
Validation accuracy: 0.9024, validation loss: 0.0186
Error loading image 

# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

y_pred = []
y_true = []

# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

testset = ImageFolder(os.path.join(data_dir, 'test'), transform=data_transforms['test'])



testloader = torch.utils.data.DataLoader(testset, batch_size=int(16),
                                        shuffle=False, num_workers=2)
device = "cuda:0"
# iterate over test data
for inputs, labels in testloader:
    inputs = inputs.to(device)
    output = model(inputs) # Feed Network

    output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
    y_pred.extend(output) # Save Prediction

    labels = labels.data.cpu().numpy()
    y_true.extend(labels) # Save Truth

# constant for classes
classes = ('atemoya', 'banana', 'bareland', 'cabbage', 'carrot',
        'grapes', 'guava', 'mango', 'papaya', 'pineapple',
        'pumpkin')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)
cf_matrix = confusion_matrix(y_true, y_pred, normalize = "true")
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) * 11, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sns.heatmap(df_cm, cmap='Greens', annot=True)
plt.savefig('output.png')